In [1]:
import heapq as hp
import urllib.request as urllib_ax
import os
from collections import defaultdict
from bitarray import bitarray
import _pickle as pickle


# Download the file if need be:
def download_file(url, filename):
    if not os.path.exists(filename):
        urllib_ax.urlretrieve(url + filename, filename)


# build a frequency table:
def build_freq(filename):
    freq = defaultdict(int)
    with open(filename,'r',encoding='utf-8-sig') as f:
        for line in f:
            for char in line:
                freq[char] += 1
    total = float(sum(freq.values()))
    return {char: count / total for (char, count) in freq.items()}


# Now build the Huffman encoding:    
class hNode:
    def __init__(self,char, freq):
        self.l=None
        self.r=None
        self.freq=freq 
        self.char=char 
        
def buildhuff(minheap):
    n=len(minheap)
    minheap.sort(key=lambda t: t[0])
    for _ in range(n-1):
        x=minheap.pop(0)[1]
        y=minheap.pop(0)[1]
        z=hNode(None,None)
        z.l=x
        z.r=y
        z.freq=x.freq+y.freq
        minheap.insert(0,(z.freq,z))
        minheap.sort(key=lambda t: t[0])
    return minheap.pop(0)[1]  

def walk_tree(node, empty, bit=""):
    if type(node.l) is hNode:
        walk_tree(node.l,empty,bit+"0")
    else:
        empty.append((node.char,bitarray(bit+"0")))
    if type(node.r) is hNode:
        walk_tree(node.r,empty,bit+"1")
    elif (type(node.r) is hNode) and (type(node.l) is hNode):
        empty.append((node.char,bitarray(bit+"1")))
    return(empty)

def encode(symb2freq):
    mheap=[(key,val) for val,key in symb2freq.items()]
    nodes=[(x[0],hNode(x[1],x[0])) for x in mheap]
    root=buildhuff(nodes)
    lib=walk_tree(root,[])
    return {char:key for (char,key) in lib}

# Now compress the file:
def compress(filename, encoding, compressed_name=None):
    if compressed_name is None:
        compressed_name = filename + ".huff"
    output = bitarray()
    with open(filename, 'r', encoding='utf-8-sig') as f:
        for line in f:
            for char in line:
                output.extend(encoding[char])
    N = len(output)
    with open(compressed_name, 'wb') as f:
        pickle.dump(N, f)
        pickle.dump(encoding, f)
        output.tofile(f)


# Now decompress the file:
def decompress(filename, decompressed_name=None):
    if decompressed_name is None:
        decompressed_name = filename + ".dehuff"
    with open(filename, 'rb') as f:
        N = pickle.load(f)
        encoding = pickle.load(f)
        bits = bitarray()
        bits.fromfile(f)
        bits = bits[:N]

    # Totally cheating here and using a builtin method:
    output = bits.decode(encoding)

    output = "".join(output).encode('utf-8-sig')
    with open(decompressed_name, 'wb') as f:
        f.write(output)


url = "http://www.gutenberg.org/files/100/"
filename = "100-0.txt"

download_file(url, filename)
freq = build_freq(filename)
encoding=encode(freq)
encoding

{'\t': bitarray('001010001000111001111010'),
 '\n': bitarray('001000'),
 ' ': bitarray('010'),
 '!': bitarray('0010100000'),
 '"': bitarray('00101000100000'),
 '#': bitarray('001010001000111001111100'),
 '$': bitarray('00101000100011100111010'),
 '%': bitarray('00101000100011100110000'),
 '&': bitarray('101011001100010010'),
 "'": bitarray('101100100'),
 '(': bitarray('1010110011001010'),
 ')': bitarray('1010110011001000'),
 '*': bitarray('101011001100010000'),
 ',': bitarray('1010010'),
 '-': bitarray('11011011110'),
 '.': bitarray('1001110'),
 '/': bitarray('0010100010001110000'),
 '0': bitarray('1010110011000110'),
 '1': bitarray('101011001101110'),
 '2': bitarray('001010001001000'),
 '3': bitarray('1010110011011000'),
 '4': bitarray('1010110011011010'),
 '5': bitarray('0010100010010100'),
 '6': bitarray('1010110011000010'),
 '7': bitarray('10101100110001010'),
 '8': bitarray('1010110011000000'),
 '9': bitarray('0010100010001100'),
 ':': bitarray('110111011100'),
 ';': bitarray('001

In [2]:
encoding = encode(freq)
compress(filename, encoding)
decompress(filename + ".huff")
# Do you get identical files?

In [7]:
from bitarray import bitarray
class hNode:
    def __init__(self,char, freq):
        self.l=None
        self.r=None
        self.freq=freq 
        self.char=char 
        
def buildhuff(minheap):
    n=len(minheap)
    hp.heapify(minheap)
    for _ in range(n-1):
        x=minheap.pop(0)[1]
        y=minheap.pop(0)[1]
        z=hNode(None,None)
        z.l=x
        z.r=y
        z.freq=x.freq+y.freq
        minheap.insert(0,(z.freq,z))
        minheap.sort(key=lambda t: t[0])
    return minheap.pop(0)[1]

def walk_tree(node, empty, bit=""):
    if type(node.l) is hNode:
        walk_tree(node.l,empty,bit+"0")
    else:
        empty.append((node.char,bitarray(bit+"0")))
    if type(node.r) is hNode:
        walk_tree(node.r,empty,bit+"1")
    elif (type(node.r) is hNode) and (type(node.l) is hNode):
        empty.append((node.char,bitarray(bit+"1")))
    return(empty)

s = 'mississippi'
d = defaultdict(int)
for k in s:
     d[k] += 1
        
l=[(key,val) for val,key in d.items()]
l=[(x[0],hNode(x[1],x[0])) for x in l]

root= buildhuff(l)
x=walk_tree(root,empty=[])
print(x)


[('i', bitarray('00')), ('m', bitarray('1000')), ('p', bitarray('1010')), ('s', bitarray('110'))]
